In [ ]:
! [ -d "oxford_pets/images" ] && echo "skipping" || (wget -nc --no-check-certificate https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz -O oxford_pets.tar.gz && mkdir -p oxford_pets && tar -xzf oxford_pets.tar.gz -C "oxford_pets" && rm oxford_pets.tar.gz)
#!rm -f "PetImages/Dog/11702.jpg" # Remove broken image bar.tar.gz -C foo
#!rm -f "PetImages/Cat/666.jpg" # Remove broken image

In [ ]:
import os
import shutil
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import pathlib

f_ = lambda f: '_'.join(pathlib.Path(f).stem.split('_')[:-1]).lower()

In [ ]:
from sap_computer_vision.datasets import image_folder as imgf
import pathlib, json

dataset_folder = 'oxford_pets/'

datasets, class_names = split_and_register('oxford_pets_v3',
                                           base_dir=dataset_folder,
                                           splits={'test': 0.1, 'validation': 0.1, 'train': 0.8},
                                           extract_class_func=lambda f: '_'.join(pathlib.Path(f).stem.split('_')[:-1]).lower())

In [ ]:
print({k: len(v) for k, v in datasets.items()})

In [ ]:
from sap_computer_vision import setup_loggers, get_cfg, get_config_file
import numpy as np

out_dir = 'image_classification_model'
if pathlib.Path(out_dir).exists():
    shutil.rmtree(out_dir)

setup_loggers(out_dir)

cfg = get_cfg()
cfg.merge_from_file(get_config_file('Base-EarlyStopping'))
cfg.merge_from_file(get_config_file('Base-Evaluation'))
cfg.merge_from_file(get_config_file('ImageClassifier/FPN-Resnet50'))

cfg.OUTPUT_DIR = out_dir
cfg.DATASETS.TRAIN = ('oxford_pets_train', )
cfg.DATASETS.TEST = ('oxford_pets_validation', )

cfg.SOLVER.MAX_ITER = 10000
cfg.SOLVER.BASE_LR = 0.00025

cfg.SOLVER.GAMMA = float(np.sqrt(0.1))
cfg.SOLVER.EARLY_STOPPING.ENABLED = False

cfg.SOLVER.WARMUP_ITERS = max(int(0.01 * cfg.SOLVER.MAX_ITER), 200)
cfg.SOLVER.STEPS = [cfg.SOLVER.MAX_ITER * p for p in (0.15, 0.375, 0.5, 0.75, 0.9)]
for aug in ['RANDOM_LIGHTING', 'RANDOM_BRIGHTNESS', 'RANDOM_SATURATION', 'RANDOM_CONTRAST', 'RANDOM_ROTATION', 'CROP']:
    if cfg.INPUT.get(aug, None) is not None:
        cfg.INPUT[aug].ENABLED = True

cfg.SOLVER.IMS_PER_BATCH = 16
cfg.SOLVER.IMS_PER_BATCH_EVAL = 16

cfg.MODEL.IMAGE_CLASSIFIER.NUM_CLASSES = len(class_names)
#cfg.MODEL.FEATURE_EXTRACTION.POOL_BACKBONE_FEATURES = 'average' 
cfg.MODEL.FEATURE_EXTRACTION.INTERMEDIATE_SIZE = []
cfg.DATALOADER.NUM_WORKERS = 10
cfg.TEST.EVAL_PERIOD = int(cfg.SOLVER.MAX_ITER * 0.05)

In [ ]:
import os
out_dir = pathlib.Path(out_dir)
out_dir.mkdir(parents=True, exist_ok=True)
with (out_dir / 'used_config.yaml').open('w') as stream:
    stream.write(cfg.dump())

In [ ]:
from sap_computer_vision.engine import ImageClassificationTrainer

In [ ]:
trainer = ImageClassificationTrainer(cfg)
trainer.resume_or_load(resume=False)

In [ ]:
trainer.train()

In [ ]:
cfg.DATASETS.TEST = ('oxford_pets_test', )
metrics = trainer.test(cfg, trainer.model)
print(json.dumps(metrics, indent=2))

In [ ]:
dl_test = trainer.build_train_loader(cfg)

In [ ]:
for batch in dl_test:
    break

In [ ]:
print(batch)